# Datos meteorologicos de Bolivia - cochabamba
Este dataset viene por parte de SENAMHI, generado desde 01/01/2011-09/01/2025 con formato MM-DD-YYYY
con las estaciones SARCO del departamento de cochabamba.

In [ ]:
import polars as pl
import  matplotlib.pyplot as plt 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [ ]:
df = pl.read_excel("tiempo.xlsx")
df

In [ ]:
cont_null = df.null_count()
cont_null = cont_null.row(0)
cont_null

In [ ]:
remove_col = []
for i, col in enumerate(df.columns):
    if(cont_null[i] > 2000):
        remove_col.append(col)

In [ ]:
remove_col = remove_col+["gestion", "dia", "mes", "estacion", "latitud","longitud", "altura", "ETo"]
df = df.drop(remove_col)

In [ ]:
df

In [ ]:
df = df.with_columns(pl.all().cast(pl.Float64))
df

In [ ]:
medians = df.median()
medians = medians.row(0)
medians

In [ ]:
for i, col in enumerate(df.columns):
    df = df.fill_null(medians[i]).with_columns(pl.col(col))

In [ ]:
df

In [ ]:
df.null_count()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df["\"Humedad Relativa Máxima\""], df["Precipitación"])
ax.set_ylabel("Precipitacion")
ax.set_xlabel("Humedad Relativa")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df["\"Humedad Relativa Máxima\""], df["\"Temperatura Máxima\""])
ax.set_ylabel("Temperatura maxima")
ax.set_xlabel("Humedad Relativa")
plt.show()

In [ ]:
data_y = df["Precipitación"].to_numpy().flatten()
data_y

In [ ]:
df.drop_in_place("Precipitación")
df

In [ ]:
data_x = df.to_numpy()
data_x

In [ ]:
def prepare_dataset(data: np.ndarray) -> tuple[np.ndarray,np.ndarray]:
    porcen = (int) (0.8*len(data))
    data_test = data[porcen:]
    data_train = data[:porcen]
    return data_train, data_test

In [ ]:
x_train, x_test = prepare_dataset(data_x)
y_train, y_test = prepare_dataset(data_y)

In [ ]:
lin_reg = LinearRegression(positive=True)
lin_reg.fit(x_train,y_train)

In [ ]:
y_pred = lin_reg.predict(x_test)

In [ ]:
mean_squared_error(y_test, y_pred)

Probamos con datos actuales

In [ ]:
df[0]

In [ ]:
dato_actual = np.array([[33.0,23.0,33.0,19.0,25.0,23.4,11.0,2.0,0.0]])


In [ ]:
prediccion = lin_reg.predict(dato_actual)
prediccion